In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Air Quality Prediction - Modeling\n",
    "\n",
    "## Tujuan\n",
    "Notebook ini bertujuan untuk melatih model machine learning untuk memprediksi AQI.\n",
    "\n",
    "## Langkah-langkah:\n",
    "1. Split data menjadi train dan test\n",
    "2. Definisikan model-model yang akan diuji\n",
    "3. Lakukan training model\n",
    "4. Tuning hyperparameter dengan GridSearchCV/RandomizedSearchCV\n",
    "5. Evaluasi model di data validasi\n",
    "6. Simpan model terbaik"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Import libraries\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV\n",
    "from sklearn.linear_model import LinearRegression, Ridge, Lasso\n",
    "from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor\n",
    "from sklearn.svm import SVR\n",
    "from xgboost import XGBRegressor\n",
    "from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error\n",
    "import joblib\n",
    "import warnings\n",
    "\n",
    "warnings.filterwarnings('ignore')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load engineered data\n",
    "engineered_data = pd.read_csv('data/processed/air_quality_engineered.csv', index_col='date', parse_dates=True)\n",
    "print(\"Engineered data shape:\", engineered_data.shape)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Split data menjadi fitur dan target\n",
    "X = engineered_data.drop('AQI', axis=1)\n",
    "y = engineered_data['AQI']\n",
    "\n",
    "# Split menjadi train dan test (80:20)\n",
    "X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)\n",
    "print(\"Train shape:\", X_train.shape)\n",
    "print(\"Test shape:\", X_test.shape)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Daftar model yang akan diuji\n",
    "models = {\n",
    "    'Linear Regression': LinearRegression(),\n",
    "    'Ridge': Ridge(),\n",
    "    'Lasso': Lasso(),\n",
    "    'Random Forest': RandomForestRegressor(random_state=42),\n",
    "    'Gradient Boosting': GradientBoostingRegressor(random_state=42),\n",
    "    'XGBoost': XGBRegressor(random_state=42)\n",
    "}"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Training dan evaluasi model dasar\n",
    "results = {}\n",
    "for name, model in models.items():\n",
    "    model.fit(X_train, y_train)\n",
    "    y_pred = model.predict(X_test)\n",
    "    \n",
    "    # Hitung metrik\n",
    "    mse = mean_squared_error(y_test, y_pred)\n",
    "    rmse = np.sqrt(mse)\n",
    "    mae = mean_absolute_error(y_test, y_pred)\n",
    "    r2 = r2_score(y_test, y_pred)\n",
    "    \n",
    "    results[name] = {\n",
    "        'MSE': mse,\n",
    "        'RMSE': rmse,\n",
    "        'MAE': mae,\n",
    "        'R2': r2\n",
    "    }\n",
    "    \n",
    "    print(f\"{name} - RMSE: {rmse:.4f}, MAE: {mae:.4f}, R2: {r2:.4f}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Tampilkan hasil dalam tabel\n",
    "results_df = pd.DataFrame(results).T\n",
    "display(results_df)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Hyperparameter tuning untuk model terbaik (misalnya Random Forest dan XGBoost)\n",
    "\n",
    "# Tuning Random Forest\n",
    "param_grid_rf = {\n",
    "    'n_estimators': [100, 200, 300],\n",
    "    'max_depth': [None, 10, 20, 30],\n",
    "    'min_samples_split': [2, 5, 10],\n",
    "    'min_samples_leaf': [1, 2, 4]\n",
    "}\n",
    "\n",
    "rf = RandomForestRegressor(random_state=42)\n",
    "grid_search_rf = GridSearchCV(rf, param_grid_rf, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)\n",
    "grid_search_rf.fit(X_train, y_train)\n",
    "\n",
    "best_rf = grid_search_rf.best_estimator_\n",
    "print(\"Best Random Forest parameters:\", grid_search_rf.best_params_)\n",
    "\n",
    "# Tuning XGBoost\n",
    "param_grid_xgb = {\n",
    "    'n_estimators': [100, 200, 300],\n",
    "    'learning_rate': [0.01, 0.05, 0.1],\n",
    "    'max_depth': [3, 5, 7],\n",
    "    'subsample': [0.8, 0.9, 1.0],\n",
    "    'colsample_bytree': [0.8, 0.9, 1.0]\n",
    "}\n",
    "\n",
    "xgb = XGBRegressor(random_state=42)\n",
    "grid_search_xgb = RandomizedSearchCV(xgb, param_grid_xgb, n_iter=20, cv=5, \n",
    "                                    scoring='neg_mean_squared_error', n_jobs=-1, random_state=42)\n",
    "grid_search_xgb.fit(X_train, y_train)\n",
    "\n",
    "best_xgb = grid_search_xgb.best_estimator_\n",
    "print(\"Best XGBoost parameters:\", grid_search_xgb.best_params_)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Evaluasi model terbaik\n",
    "best_models = {\n",
    "    'Random Forest (tuned)': best_rf,\n",
    "    'XGBoost (tuned)': best_xgb\n",
    "}\n",
    "\n",
    "final_results = {}\n",
    "for name, model in best_models.items():\n",
    "    y_pred = model.predict(X_test)\n",
    "    \n",
    "    mse = mean_squared_error(y_test, y_pred)\n",
    "    rmse = np.sqrt(mse)\n",
    "    mae = mean_absolute_error(y_test, y_pred)\n",
    "    r2 = r2_score(y_test, y_pred)\n",
    "    \n",
    "    final_results[name] = {\n",
    "        'MSE': mse,\n",
    "        'RMSE': rmse,\n",
    "        'MAE': mae,\n",
    "        'R2': r2\n",
    "    }\n",
    "    \n",
    "    print(f\"{name} - RMSE: {rmse:.4f}, MAE: {mae:.4f}, R2: {r2:.4f}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Bandingkan hasil model\n",
    "final_results_df = pd.DataFrame(final_results).T\n",
    "display(final_results_df)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Simpan model terbaik\n",
    "joblib.dump(best_xgb, 'models/trained_models/xgboost_model.pkl')\n",
    "print(\"Best model (XGBoost) saved to 'models/trained_models/xgboost_model.pkl'\")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.5"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}